# HCMUS-UNIV-WebScience-VisualizeData

## PROJECT 2: MỐI QUAN HỆ CỦA DỮ LIỆU


### I. Chuẩn bị môi trường:

In [59]:
import sqlite3
import pandas as pd
import json

In [60]:
def fetch_value(sqlCommand):
    conn = sqlite3.connect("data.db",check_same_thread=False)
    ret = pd.read_sql_query(sqlCommand, conn)
    conn.close()
    return ret

# Query dữ liệu
def query_sql(sql:str):
    listData = fetch_value(sql)
    return listData

# Lấy bài viết theo website.
def get_bai_viet(website:str):
    return query_sql(f"select * from baiviet where website='{website}'")

class Utils:
    def convertHMtoFloat(HM):
        h,m = HM.split(":")
        return int(h)+(float(m)/60)

### I. Tiền xử lý dữ liệu

 #### Yêu cầu: đưa các field về dạng chuẩn chung:

1. active_time: thời gian hoạt động, đưa về dạng sau:

```
    {
        <tên ngày trong tuần>:[ <thời gian mở cửa>,<thời gian đóng cửa> ]
    }

    *lowercase
    *thời gian đưa về float: 8h30 => 8.5
```
    
- ví dụ:

```
    {
        "thứ 2": [7,21.5],
        ...,
        "chủ nhật": [8,22.3]
    }
```

2. other_service: các dịch vụ khác, đưa về dạng sau:

```
    [
        <tên dịch vụ>,
        <tên dịch vụ>,
        ...,
    ]
    
    *lowercase
```
    
- ví dụ:

```
    [
        "có wifi"
        "giao hàng",
        ...,
    ]
```

#### I.1 Foody

In [61]:
def preprocessing_Foody():
    # Code here
    data = get_bai_viet("Foody")
    active_time = data["active_time"]
    other_service = data["other_service"]
    format_time = []
    format_other = []
    for time in active_time:
        #time = time.open + ',' + time.close
        time = json.loads(time)
        time_format = []
        time_format.append(time['open'])
        time_format.append(time['close'])
        x = {
            "thứ 2":time_format,
            "thứ 3":time_format,
            "thứ 4":time_format,
            "thứ 5":time_format,
            "thứ 6":time_format,
            "thứ 7":time_format,
            "chủ nhật":time_format,
            }
        time = json.dumps(x)
        dict_time = json.loads(time)
        format_time.append(dict_time)
    data['active_time'] = format_time
    for other in other_service:
        other = other.lower()
        format_other.append(other)
    data['other_service'] = format_other
    return data

#### I.2 ShopeeFood

In [62]:
def preprocessing_ShopeeFood():
    def maping(name):
        if name=="monday":
            return "thứ 2"
        if name=="tuesday":
            return "thứ 3"
        if name=="wednesday":
            return "thứ 4"
        if name=="thursday":
            return "thứ 5"
        if name=="friday":
            return "thứ 6"
        if name=="saturday":
            return "thứ 7"
        if name=="sunday":
            return "chủ nhật"

    def convert_active_time(HM):
        pre_format = {}
        at_js = json.loads(HM)
        atl = list(at_js)
        for key in atl:
            open,close = at_js[key].split("-")
            pre_format[maping(key)] = [Utils.convertHMtoFloat(open),Utils.convertHMtoFloat(close)]
        return pre_format

    data = get_bai_viet("ShopeeFood")
    data["active_time"] = data["active_time"].apply(convert_active_time)

    return data

#### I.3 GoogleReview

In [63]:
def preprocessing_GoogleReview():
    # Code here
    return pd.DataFrame()

#### I.4 Tổng hợp lại dữ liệu tiền xử lý

In [64]:
# DATA = pd.concat([preprocessing_Foody(),preprocessing_ShopeeFood(),preprocessing_GoogleReview()])
preprocessing_Foody()
# DATA.describe()

[{'thứ 2': [18, 23], 'thứ 3': [18, 23], 'thứ 4': [18, 23], 'thứ 5': [18, 23], 'thứ 6': [18, 23], 'thứ 7': [18, 23], 'chủ nhật': [18, 23]}, {'thứ 2': [11, 23], 'thứ 3': [11, 23], 'thứ 4': [11, 23], 'thứ 5': [11, 23], 'thứ 6': [11, 23], 'thứ 7': [11, 23], 'chủ nhật': [11, 23]}, {'thứ 2': [10, 2], 'thứ 3': [10, 2], 'thứ 4': [10, 2], 'thứ 5': [10, 2], 'thứ 6': [10, 2], 'thứ 7': [10, 2], 'chủ nhật': [10, 2]}, {'thứ 2': [11, 22], 'thứ 3': [11, 22], 'thứ 4': [11, 22], 'thứ 5': [11, 22], 'thứ 6': [11, 22], 'thứ 7': [11, 22], 'chủ nhật': [11, 22]}, {'thứ 2': [10, 21], 'thứ 3': [10, 21], 'thứ 4': [10, 21], 'thứ 5': [10, 21], 'thứ 6': [10, 21], 'thứ 7': [10, 21], 'chủ nhật': [10, 21]}, {'thứ 2': [10, 22], 'thứ 3': [10, 22], 'thứ 4': [10, 22], 'thứ 5': [10, 22], 'thứ 6': [10, 22], 'thứ 7': [10, 22], 'chủ nhật': [10, 22]}, {'thứ 2': ['10:00', '23:00'], 'thứ 3': ['10:00', '23:00'], 'thứ 4': ['10:00', '23:00'], 'thứ 5': ['10:00', '23:00'], 'thứ 6': ['10:00', '23:00'], 'thứ 7': ['10:00', '23:00'], 'ch

[]
['cho mua về', 'giữ xe máy miễn phí']
['có wifi']
['cho mua về']
['có wifi', 'có máy lạnh & điều hòa', 'cho mua về']
['có wifi', 'có máy lạnh & điều hòa']
[]
['có giao hàng', 'cho mua về', 'giữ xe máy miễn phí']
[]
['có bàn ngoài trời', 'giữ xe máy miễn phí']
[]
['có wifi', 'có máy lạnh & điều hòa', 'nên đặt trước', 'có phòng riêng', 'có nhạc sống']
[]
['có wifi', 'có bàn ngoài trời']
[]
['có máy lạnh & điều hòa', 'trả bằng thẻ', 'có chỗ đậu ôtô', 'nên đặt trước', 'có phòng riêng', 'giữ xe máy miễn phí', 'có khu vực hút thuốc']
['có wifi', 'cho mua về', 'giữ xe máy miễn phí']
['có wifi', 'có máy lạnh & điều hòa', 'cho mua về', 'trả bằng thẻ', 'có nhạc sống']
[]
['có wifi', 'trả bằng thẻ', 'nên đặt trước']
[]
['có wifi', 'có máy lạnh & điều hòa', 'có chỗ đậu ôtô', 'nên đặt trước', 'có phòng riêng', 'giữ xe máy miễn phí', 'có xuất hóa đơn đỏ', 'có hỗ trợ người khuyết tật']
['có wifi', 'có máy lạnh & điều hòa', 'cho mua về', 'trả bằng thẻ', 'có chỗ đậu ôtô']
[]
[]
['có wifi', 'có máy l

[]
['có wifi', 'có máy lạnh & điều hòa', 'nên đặt trước']
['cho mua về']
[]
['có wifi', 'cho mua về', 'trả bằng thẻ', 'nên đặt trước', 'có phòng riêng', 'có xuất hóa đơn đỏ', 'có nhạc sống']
['cho mua về']
[]
['có wifi', 'có máy lạnh & điều hòa', 'có khu vực hút thuốc']
['cho mua về', 'giữ xe máy miễn phí']
[]
['có wifi', 'có máy lạnh & điều hòa', 'trả bằng thẻ']
['cho mua về']
['có wifi', 'có máy lạnh & điều hòa', 'cho mua về']
[]
['cho mua về']
['có wifi', 'có bàn ngoài trời']
['cho mua về', 'giữ xe máy miễn phí']
['có wifi', 'có máy lạnh & điều hòa', 'cho mua về', 'nên đặt trước']
['có wifi', 'cho mua về', 'trả bằng thẻ', 'nên đặt trước', 'giữ xe máy miễn phí', 'có khu vực hút thuốc']
['cho mua về']
['cho mua về']
['cho mua về', 'giữ xe máy miễn phí']
['nên đặt trước']
[]
['có wifi', 'có bàn ngoài trời']
['cho mua về', 'nên đặt trước']
[]
['cho mua về', 'có bàn ngoài trời', 'giữ xe máy miễn phí']
[]
['cho mua về']
['có wifi', 'có giao hàng', 'cho mua về', 'có chỗ đậu ôtô', 'giữ xe m


['có máy lạnh & điều hòa']
['có wifi', 'có giao hàng', 'có máy lạnh & điều hòa', 'cho mua về', 'có chỗ đậu ôtô', 'nên đặt trước', 'có bàn ngoài trời', 'giữ xe máy miễn phí']
['giữ xe máy miễn phí']
['cho mua về', 'giữ xe máy miễn phí']
['cho mua về', 'giữ xe máy miễn phí']
[]
['có giao hàng', 'cho mua về', 'giữ xe máy miễn phí']
['có máy lạnh & điều hòa', 'giữ xe máy miễn phí']
[]
['có giao hàng', 'có máy lạnh & điều hòa', 'cho mua về', 'có chỗ đậu ôtô', 'nên đặt trước', 'có bàn ngoài trời', 'có phòng riêng', 'có chỗ chơi cho trẻ em', 'giữ xe máy miễn phí', 'có khu vực hút thuốc', 'có thẻ thành viên', 'có hỗ trợ hội thảo']
['có giao hàng', 'cho mua về', 'có bàn ngoài trời', 'giữ xe máy miễn phí', 'có khu vực hút thuốc']
['có wifi', 'có giao hàng', 'cho mua về', 'có chỗ đậu ôtô', 'có bàn ngoài trời', 'có chỗ chơi cho trẻ em', 'giữ xe máy miễn phí', 'có khu vực hút thuốc', 'có thẻ thành viên']
['cho mua về', 'có bàn ngoài trời']
['cho mua về', 'có bàn ngoài trời', 'giữ xe máy miễn phí']

['có giao hàng', 'nên đặt trước']
['có giao hàng', 'nên đặt trước']
[]
[]
[]
[]
[]
['có wifi', 'có giao hàng', 'có máy lạnh & điều hòa', 'cho mua về', 'có bàn ngoài trời', 'có phòng riêng', 'có chỗ chơi cho trẻ em', 'giữ xe máy miễn phí']
['có wifi', 'có giao hàng']
['có giao hàng', 'cho mua về', 'trả bằng thẻ', 'có khu vực hút thuốc']
['có wifi', 'có giao hàng', 'cho mua về', 'trả bằng thẻ', 'có chiếu bóng đá']
['có giao hàng', 'cho mua về', 'giữ xe máy miễn phí']
['có giao hàng', 'cho mua về', 'nên đặt trước']
['có giao hàng']
['có wifi', 'có giao hàng', 'có máy lạnh & điều hòa', 'cho mua về', 'trả bằng thẻ', 'nên đặt trước', 'giữ xe máy miễn phí']
['có wifi', 'có giao hàng', 'có máy lạnh & điều hòa', 'cho mua về', 'có phòng riêng', 'giữ xe máy miễn phí', 'có khu vực hút thuốc']
[]
['có giao hàng', 'cho mua về', 'giữ xe máy miễn phí']
['có giao hàng']
['cho mua về']
['có giao hàng', 'cho mua về']
['cho mua về']
['có giao hàng', 'cho mua về', 'nên đặt trước']
['có wifi', 'có giao hàng

,website_id,website,url,full_name,phone,district,rate,rate_count,favorite,active_time,price_from,price_to,other_service
0,Foody_107609,Foody,https://www.foody.vn/ho-chi-minh/the-log-resta...,The LOG Restaurant,None,1,7.896,251,887,"{'thứ 2': [18, 23], 'thứ 3': [18, 23], 'thứ 4'...",200000.0,1500000.0,"[""c\u00f3 wifi"", ""c\u00f3 m\u00e1y l\u1ea1nh &..."
1,Foody_91179,Foody,https://www.foody.vn/ho-chi-minh/nha-hang-sora...,Sorae Restaurant - Lounge,None,1,8.168,251,701,"{'thứ 2': [11, 23], 'thứ 3': [11, 23], 'thứ 4'...",300000.0,1000000.0,"[""c\u00f3 wifi"", ""c\u00f3 m\u00e1y l\u1ea1nh &..."
2,Foody_20860,Foody,https://www.foody.vn/ho-chi-minh/eon-heli-bar-...,EON 52 Heli Bar - Bitexco Tower,None,1,7.822,110,306,"{'thứ 2': [10, 2], 'thứ 3': [10, 2], 'thứ 4': ...",200000.0,550000.0,"[""c\u00f3 wifi"", ""c\u00f3 m\u00e1y l\u1ea1nh &..."
3,Foody_8,Foody,https://www.foody.vn/ho-chi-minh/buffet-windso...,Buffet Hải Sản - Café Central An Đông - Khách ...,None,5,7.224,185,364,"{'thứ 2': [11, 22], 'thứ 3': [11, 22], 'thứ 4'...",449000.0,853000.0,"[""c\u00f3 wifi"", ""c\u00f3 m\u00e1y l\u1ea1nh &..."
4,Foody_94523,Foody,https://www.foody.vn/ho-chi-minh/an-duyen-nha-...,Bếp Xanh An Duyên - Nhà Hàng Chay,None,5,7.880,151,224,"{'thứ 2': [10, 21], 'thứ 3': [10, 21], 'thứ 4'...",30000.0,250000.0,"[""c\u00f3 wifi"", ""c\u00f3 m\u00e1y l\u1ea1nh &..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3022,Foody_999121,Foody,https://www.foody.vn/ho-chi-minh/nhom-nau-an-b...,Nhóm Nấu Ăn Ba Tu,None,thủ đức,0.000,0,0,"{'thứ 2': [7, 21], 'thứ 3': [7, 21], 'thứ 4': ...",100000.0,500000.0,"[""n\u00ean \u0111\u1eb7t tr\u01b0\u1edbc""]"
3023,Foody_1032768,Foody,https://www.foody.vn/ho-chi-minh/hai-thuy-cate...,Hai Thụy Catering - Cơm Văn Phòng,None,bình thạnh,0.000,0,0,"{'thứ 2': [8, 17], 'thứ 3': [8, 17], 'thứ 4': ...",30000.0,100000.0,"[""c\u00f3 giao h\u00e0ng"", ""n\u00ean \u0111\u1..."
3024,Foody_1000022520,Foody,https://www.foody.vn/ho-chi-minh/a-say-oc-hai-...,"A Sấy - Ốc, Hải Sản - Bến Vân Đồn",None,4,0.000,0,0,"{'thứ 2': [15, 21], 'thứ 3': [15, 21], 'thứ 4'...",0.0,0.0,"[""cho mua v\u1ec1"", ""n\u00ean \u0111\u1eb7t tr..."
3025,Foody_855723,Foody,https://www.foody.vn/ho-chi-minh/dich-vu-nau-a...,Dịch Vụ Nấu Ăn Hạnh Phúc,None,bình thạnh,0.000,0,0,"{'thứ 2': [7, 23], 'thứ 3': [7, 23], 'thứ 4': ...",500000.0,5000000.0,"[""c\u00f3 giao h\u00e0ng"", ""n\u00ean \u0111\u1..."


### 2. Thống kê dữ liệu

// Mô tả ...

In [65]:
# Code here

###  Phần 3,4,5